In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whic

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments, LlamaForSequenceClassification
import torch
from torch.utils.data import DataLoader
import os
import subprocess
import json

In [ ]:
# Download dataset from GitHub
repo_url = "https://github.com/ltgoslo/norec.git"
dataset_path = "./norec"
if not os.path.exists(dataset_path):
    subprocess.run(["git", "clone", repo_url, dataset_path])


In [ ]:
# Load tokenizer and model
model_name = "meta-llama/Llama-3.3"  # Example name, replace with actual model source
tokenizer = LlamaTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: meta-llama/Llama-3.3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import datasets

# Load dataset using datasets library
dataset = datasets.load_dataset("json", data_files={
    "train": os.path.join(dataset_path, "data", "train.json"),
    "dev": os.path.join(dataset_path, "data", "dev.json"),
    "test": os.path.join(dataset_path, "data", "test.json")
})

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [ ]:
# Fine-tune for causal language modeling
model_clm = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

training_args_clm = TrainingArguments(
    output_dir="./llama_norwegian_clm",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs_clm",
    logging_steps=100,
    fp16=True,
    report_to="none"
)

trainer_clm = Trainer(
    model=model_clm,
    args=training_args_clm,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer_clm.train()

# Fine-tune for sequence classification
model_cls = LlamaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: positive, neutral, negative

train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=8, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["test"], batch_size=8)

training_args_cls = TrainingArguments(
    output_dir="./llama_norwegian_cls",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5
)

trainer_cls = Trainer(
    model=model_cls,
    args=training_args_cls,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

trainer_cls.train()

ModuleNotFoundError: No module named 'datasets'